> Each time you connect, I generate a new one time pad. I also check for leaks, so there's no chance you can learn anything about my secrets! <br> Connect at ```nc socket.cryptohack.org 13370```

[NoLeak.py](/NoLeak.py)

The critical in this script is the wrong implementation in the function `xor_flag_with_otp()`, let's analyze the function.

The variable `otp` is generated from `os.urandom()`, which is cryptographically secured (You can have more information in this [link](https://crypto.stackexchange.com/questions/39186/)). So we can 'safely' remove the chance of having an attack on the PRG. (Which we will demonstrate in another attack). 

It seems the OTP here is very well-implemented, as it always created a secure OTP that has the same length as the plaintext. A complete theoretical OTP implementation, even with an additional support that it can 'remove ciphertext with leak bytes' from the flag. 

So what is the vuln?

#### **Stop some minutes and think about it!**



Actually, the additional part that can remove the leaked bytes is the vulnerability. 

```Python
    for c, p in zip(xored, flag_ord):
        assert c != p
```

First, let's see, why the ciphertext could leak some bytes from the plaintext? The function `os.urandom(n)` will give us a random bytestring with a given length $n$. Each byte has the value in range $[0, 255]$ in decimal. If `urandom` gives a byte `0` in the OTP, it could 'leak' the byte from the plaintext, as:

$$
k \oplus 0 = k
$$

Therefore, the ciphertext $c$ we are given, there are no position $i (0 <= i < n)$ that $c_i = p_i$.

However, in this challenge, we can request as many ciphertext (from the same plaintext) as we want. Also, we know that at any position, $c_i$ only can hold a value from $[0, 255]$, except the value in the plaintext. 

Therefore, we can do some eliminations based on the ciphertext we request. For example, if $p_i = 32$, this implies that $c_i \in [0, 255] \setminus \{32\}$. We will request many ciphertexts, and if we have seen all value in the set $[0, 255] \setminus \{32\}$, we can say that $p_i = 32$.

#### **Here is the implementation of the challenge**

In [1]:
# import some library
import socket
import json
import string
import base64

In [2]:
# template for the socket request
request = {"msg": "request"}
# information about the host
ip = "socket.cryptohack.org"
port = 13370
#  
result = [0]*20 # for recording the plaintext byte

In [3]:
possibleAlphabet = string.digits + string.ascii_letters + string.punctuation # the alphabet of our ciphertext
alphabet = [ord(i) for i in possibleAlphabet] # just a convertion from char -> byte

In [4]:
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((ip, port))

# Connect to our host!

In [5]:
print(client.recv(10240).decode()) # print out the first message

No leaks



In [6]:
checkFinish = []
for i in range(20):
    checkFinish.append(set(alphabet))
# this guys use for us to check which character haven't been seen in a specific location i

In [7]:
def checkAll():
    for i in result:
        if i == 0:
            return False
    return True

In [ ]:
# DON'T RUN THIS GUY IN JUPYTER!!
# You should convert the notebook into python script to print it out terminal.
while not checkAll():
    # send the request for the next ciphertext
    client.send(json.dumps(request).encode())
    # receive the res and load the json
    encByte = client.recv(10240)
    test = json.loads(encByte)
    # get the "ciphertext"
    if "ciphertext" in test:
        # get the raw bytes, through base64 decoding
        encryptedBytes = base64.b64decode(test["ciphertext"])
        # iterate through the raw bytes, for each location, remove the byte in checkFinish
        for i in range(0, len(encryptedBytes)):
            if encryptedBytes[i] in checkFinish[i]:
                checkFinish[i].remove(encryptedBytes[i])
            # if there is only one byte (i.e. the byte that doesn't appear in ciphertext, get it)
            if len(checkFinish[i]) == 1:
                result[i] = checkFinish[i].pop()
                print(result)
        # trace the progress of the cryptanalysis
        for i in range(0, len(checkFinish)):
            print(len(checkFinish[i]), end = " ")
        print()
# Bingo! You got it!
for i in result:
    print(chr(i), end = "")